In [34]:
import spacy
import pandas as pd
from spacy.matcher import Matcher
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 
import numerizer
nlp = spacy.load("en_core_web_lg")
numnlp = spacy.load("en_core_web_lg")
nlp.add_pipe("merge_entities")
nlp.add_pipe("merge_noun_chunks")
import json
from matplotlib import pyplot as plt
import cv2
from elasticsearch import Elasticsearch
import re
from nltk.corpus import wordnet as wn

In [2]:
from elasticsearch import helpers
client = Elasticsearch(
    "http://localhost:9200",
     basic_auth=("elastic", "elastic")

)
resp = client.info()

In [32]:
class question_matcher:
    def __init__(self):
        self.patterns= []
        self.q_json = self.get_questions_json()
        self.matcher = Matcher(nlp.vocab)

    def get_questions_json(self):
        with open('questions.json', 'r') as f:
            pat = json.load(f)
        return pat
    
    def define_keyword(self,question:str):
        keyword = ''
        for q in self.q_json:
            if q['keyword'] in question.lower():
                keyword = q['keyword']
                index = q['qid']
        return keyword, index

    def re_get_focus(self,question:str,ind,pattern):
        question_token = [token.text for token in nlp(question)]
        focus = ''
        focus_pos = pattern['focus']
        start = 0
        i = 0
        for token in focus_pos[ind].split('\s*'):
                if(token == 'sf'):
                    start = 1
                    i = i - 1
                    continue

                if(token == 'ef'):
                    start = 0
                    break

                if(token == ".*" and start == 1):
                    j = i
                    while(j < len(question_token)):
                        focus = focus + ' ' + question_token[j]
                        j = j + 1
                    focus = focus.strip()
                    print(focus)
                    return focus

                if(start == 1):
                    focus = focus + ' ' + question_token[i]
                i = i + 1
        focus = focus.strip()
        print(focus)
        return focus

    def re_match_pattern(self,question):
        found = 0
        doc = nlp(question)
        ltoken = [token.pos_ for token in nlp(question)]
        q_pattern = ''
        for token in doc:
            q_pattern = q_pattern + token.pos_ + ' ' 
        keyword, index = self.define_keyword(question)
        for pattern in self.q_json[index]['patterns']:
            matched_pos=0
            for pos in pattern['pos']:
                regexpos  = f'({pos})'
                matches = re.fullmatch(regexpos, q_pattern)
                if matches != None:
                    #print('Matched!')
                    # print(matches.string)
                    # print(pos)
                    # print(pattern['pattern'])
                    focus = self.re_get_focus(question,matched_pos,pattern)
                    found = 1
                    return focus, pattern['answerType']
                matched_pos = matched_pos + 1
            if found == 1:
                break
 
    def pattern_to_pos(self, patterns:list):
        pos = []
        pattern = []
        for p in patterns:
            
            itr = []
            pattern.append(p)
            for token in p.split():
                if token != '*':
                    itr.append({'POS':token})
                else:
                    itr.append({})
            pos.append(itr)
        return pos
qm = question_matcher()


In [30]:
qm.re_match_pattern("images that contain more than ten people")

Matched!
more than ten people


('more than ten people', 'entity')

In [16]:

class retrive_image:
    def __init__(self):
        self.image_ids =[]
        self.image_caption =[]

    def search(self,query):
        self.image_ids =[]
        self.image_caption =[]
        query = {
            "match": {
            "caption": query
            }
        }
        resp = client.search(index="cocomo", query=query,size=100)
        #print("Got %d Hits:" % resp['hits']['total']['value'])
        for hit in resp['hits']['hits']:
            #  print(\"%(caption)s : %(image_id)s : %(id)s\" % hit[\"_source\"])\n",
            self.image_ids.append(hit['_source']['image_id'])
            self.image_caption.append(hit['_source']['caption'])
       # self.display();
        
    def display(self):
        i = 0
        plt.figure(figsize=(25,50))
        columns = 2
        for i, id in enumerate(self.image_ids):
            plt.subplot(10, 3, i + 1)
            path = '..\\..\\Cocomo2014\\\\val2014\\\\val2014\\COCO_val2014_'+ '0'*(12 - len(str(id))) + str(id) +'.jpg'
            img = cv2.imread(path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            plt.title(self.image_caption[i])
            plt.imshow(img)
ir = retrive_image()


In [103]:
class system1:
    def __init__(self):
        qm = question_matcher()
        ir = retrive_image()
        self.image_captions = []
        self.image_ids = []
        self.data = pd.DataFrame()
    
    def search(self,question):
        focus, eat =  qm.re_match_pattern(question)
        print(focus)
        ir.search('people')
       # print(eat)
        self.image_captions = ir.image_caption
        self.image_ids = ir.image_ids
        self.data = pd.DataFrame({"caption":self.image_captions, "id":self.image_ids, "EAT":[0]*len(self.image_ids),"similarity":[0]*len(self.image_ids),"num":[0]*len(self.image_ids)})
        #print(self.data.head())
        #self.get_image_with_correct_synset(eat, focus)
        self.get_image_with_number(focus)
        self.data = self.data.sort_values(by=['EAT','num'],ascending=False)
        print(self.data.head(50))
        #self.display(self.data.head(50)['caption'],self.data.head(50)['id'])
        #print(self.data['caption'].at[5])
    
    # def creating_query(self, qid, focus):
    #     if qid < 4:
    #         return focus
    #     if qid == 4:
            

    def get_image_with_correct_synset(self, eat:str, focus:str):
        for index, row in self.data.iterrows():
            self.data.at[index, 'similarity'] = nlp(focus).similarity(nlp(row['caption']))
            for token in row['caption'].split():
                # if(token.is_stop):
                #     continue
                #print(token)
                cond = self.check_hypernym(token, eat)

                if (cond == 'True'):
                    self.data.at[index, 'EAT'] = 1
                    # break
            
            # for ent in nlp(row['caption']).ents:
            #     if(eat == 'person' and ent.label_ == 'PERSON'):
            #         self.data.at[index, 'EAT'] = 1
                    
       # return aftercap, afterids



    def get_image_with_number(self, focus:str):
        focus_head, focus_num = self.get_num_with_head(focus)
        for index, row in self.data.iterrows():
            for token in row['caption'].split():
                if(nlp(token)[0].pos_ == 'NUM'):
                    self.data.at[index, 'EAT'] = 1
               
                    fuck = str(row['caption'])
                    #print(fuck)
                    word_head, word_num =  self.get_num_with_head(fuck)
                    if(word_head == focus_head and word_num>=focus_num):
                        self.data.at[index, 'num'] = 1
                

                

                
            


    def get_num_with_head(self,s:str):
        doc = numnlp(s)
        for token in doc:
            if(token.dep_ == 'nummod'):
                num = list(numnlp(token.text)._.numerize().values())[0]
                return token.head.text, float(num)

        
    def check_hypernym(self ,word:str ,sett:str):
        word_lamma = nlp(word)[0].lemma_
        sett_lamma = nlp(sett)[0].lemma_

        if(wn.synsets(word_lamma) == []):
            return 'False'
        word_ss = wn.synsets(word_lamma)[0]

        if(wn.synsets(sett_lamma) == []):
            return 'False'
        sett_ss = set(ss.name() for ss in wn.synsets(sett_lamma))
        hypernyms_ss = {synset.name() for synset in wn.synset(word_ss.name()).hypernym_paths()[0]}
        z = sett_ss.intersection(hypernyms_ss)
        if(len(list(z)) != 0):
           # print('yezzzz')
            return 'True'
        return 'False'



    def display(self,caps,ids):
        i = 0
        plt.figure(figsize=(25,50))
        for i, id in enumerate(ids):
            plt.subplot(18, 2, i + 1)
            path = '..\\..\\Cocomo2014\\\\val2014\\\\val2014\\COCO_val2014_'+ '0'*(12 - len(str(self.data.head(50)['id'][i]))) + str(self.data.head(50)['id'][i]) +'.jpg'
            img = cv2.imread(path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            plt.title(self.data.head(50)['caption'][i])
            plt.imshow(img)
s1 = system1()

In [104]:
s1.search('images that contain more than two people')

more than two people
more than two people
                                              caption      id  EAT  \
0     Four people watching 3 other people race horses  439481    1   
2   some people are watching two people playing te...  422522    1   
7   Two people with umbrellas near other people wi...   20774    1   
14  People sitting in the grass while two people a...   88682    1   
15  Two people riding a motorcycle near a group of...  331785    1   
18  Two people on horses surrounded by a crowd of ...   94055    1   
20  many people sitting on a field with two people...  578441    1   
21  Two people in skis racing up slope with people...  538115    1   
23  Two people playing a video game with three peo...  324322    1   
25  Three people people in a kitchen preparing foo...  369259    1   
36  Two people playing tennis on a tennis court wh...   36226    1   
39  Two people are playing wii together and laughi...  140332    1   
42  Two people sit on a bench with pigeons and o

In [62]:
sy = {'s': 'l', 'm0':'ll'}
print(list(sy.values()))

['l', 'll']


In [ ]:
def get_entity_eat(entity):
    person = wn.synsets('person')
    person_ss = set(ss.name() for ss in wn.synsets('person'))
    location_ss = set(ss.name() for ss in wn.synsets('location'))


In [69]:
def get_num_with_head(s:str):
    doc = numnlp(s)
    for token in doc:
        if(token.dep_ == 'nummod'):
            num = list(numnlp(token.text)._.numerize().values())[0]
            return token.head.text, float(num)
get_num_with_head("more than three other people and ten dogs")

IndexError: list index out of range

In [49]:
doc = numnlp("more than three other people")
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
num = doc._.numerize()

more amod three NUM []
than quantmod three NUM []
three nummod people NOUN [more, than]
other amod people NOUN []
people ROOT people NOUN [three, other]


In [66]:
doc = nlp('three people and dogs')
doc._.numerize()

{}

In [ ]:
def synonym_antonym_extractor(phrase):
     from nltk.corpus import wordnet
     synonyms = []
     antonyms = []

     for syn in wordnet.synsets(phrase):
          for l in syn.lemmas():
               synonyms.append(l.name())
               if l.antonyms():
                    antonyms.append(l.antonyms()[0].name())
     print(set(synonyms))
     print(set(antonyms))
synonym_antonym_extractor(phrase="house")